In [ ]:
from RandomTesting.__RandomTesting__ import Random_Testing, calculate_extract_loss
from tqdm import tqdm
from sklearn import neighbors
from pykrige.ok import OrdinaryKriging
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
import numpy as np
import warnings
import os
import pandas as pd
import gc


warnings.filterwarnings('ignore')
os.environ["CUDA_VISIBLE_DEVICES"] = ""
tf.config.optimizer.set_jit(True)

# AIR_Kriging_Align

In [ ]:
air1718 = pd.read_csv('air-201718.csv')
air1718_arr = air1718[7075495:]
air19 = pd.read_csv('air-2019.csv')
air171819_arr = np.concatenate([air1718_arr,air19])

first_lst = []
count=0
for data_nb in tqdm(range(len(air171819_arr))):
    if (air171819_arr[data_nb][0][-8:] == '08:00:00') or (air171819_arr[data_nb][0][-8:] == '20:00:00'):
        count+=1
        first_lst.append(air171819_arr[data_nb])
print(np.array(first_lst).shape)
df = pd.DataFrame(data=np.array(first_lst),columns=['dt',"lat", "lon",'pm25','pm10','temperature','humidity'])
df.to_csv('Align_AIR_Training.csv',index=False)

df = pd.read_csv('Align_AIR_Training.csv')

dts=df.dt.unique()

air_kriging = []
grid_lon = np.array(np.arange(204),dtype='float32')
grid_lat = np.array(np.arange(348),dtype='float32') 

for dt in tqdm(dts[:]):
    one_dt=df[df.dt==dt]

    x_train = one_dt[['lat', 'lon']]
    y_train = one_dt.iloc[:, 3:4] #pm2.5

    matrix_lon_list = [];matrix_lat_list = []; value_list=[]
    for i in range(len(x_train)):
        if (np.array(x_train)[i][0] > 21.87) and (np.array(x_train)[i][0] < 25.35) and (np.array(x_train)[i][1] > 120) and (np.array(x_train)[i][1] < 122.04) and (np.array(y_train)[i][0] <=300):
            matrix_lon = int(round((np.array(x_train)[i][1]-120)/((122.04-120)/204),0))
            matrix_lat = int(round((np.array(x_train)[i][0]-21.87)/((25.35-21.87)/348),0))
            matrix_lon_list.append(matrix_lon)
            matrix_lat_list.append(matrix_lat)
            value_list.append(np.array(y_train)[i][0])

    matrix_lon_array = np.array(matrix_lon_list,dtype='float32')
    matrix_lat_array = np.array(matrix_lat_list,dtype='float32')       
    train_x = np.array(np.vstack((matrix_lon_array,matrix_lat_array)).T,dtype='float32')    
    train_y = np.array(value_list,dtype='float32')


    OK = OrdinaryKriging(train_x[:,0], train_x[:,1], train_y, variogram_model='linear',
                         verbose=False, nlags=50, weight=True, exact_values=False,pseudo_inv=True)

    values, ss1 = OK.execute('grid', grid_lon, grid_lat)
    air_kriging.append(values)

    plt.imshow(values,origin='lower')
    plt.title(str(dt))
    plt.colorbar()
    plt.show()
    
    del matrix_lon_array; del matrix_lat_array; del train_x; del train_y; del OK; del values; del ss1        

print(len(air_kriging))

In [ ]:
air20_arr = np.array(pd.read_csv('air-2020.csv'))
first_lst = []
count=0
for data_nb in tqdm(range(len(air20_arr))):
    if (air20_arr[data_nb][0][-8:] == '08:00:00') or (air20_arr[data_nb][0][-8:] == '20:00:00'):
        count+=1
        first_lst.append(air20_arr[data_nb])
print(np.array(first_lst).shape)
df = pd.DataFrame(data=np.array(first_lst),columns=['dt',"lat", "lon",'pm25','pm10','temperature','humidity'])
df.to_csv('Align_AIR_Testing.csv',index=False)

df = pd.read_csv('Align_AIR_Testing.csv')

dts=df.dt.unique()

# air_kriging = []
grid_lon = np.array(np.arange(204),dtype='float32')
grid_lat = np.array(np.arange(348),dtype='float32') 

for dt in tqdm(dts[162:163]):
    one_dt=df[df.dt==dt]

    x_train = one_dt[['lat', 'lon']]
    y_train = one_dt.iloc[:, 3:4] #pm2.5

    matrix_lon_list = [];matrix_lat_list = []; value_list=[]
    for i in range(len(x_train)):
        if (np.array(x_train)[i][0] > 21.87) and (np.array(x_train)[i][0] < 25.35) and (np.array(x_train)[i][1] > 120) and (np.array(x_train)[i][1] < 122.04) and (np.array(y_train)[i][0] <=300):
            matrix_lon = int(round((np.array(x_train)[i][1]-120)/((122.04-120)/204),0))
            matrix_lat = int(round((np.array(x_train)[i][0]-21.87)/((25.35-21.87)/348),0))
            matrix_lon_list.append(matrix_lon)
            matrix_lat_list.append(matrix_lat)
            value_list.append(np.array(y_train)[i][0])

    matrix_lon_array = np.array(matrix_lon_list,dtype='float32')
    matrix_lat_array = np.array(matrix_lat_list,dtype='float32')       
    train_x = np.array(np.vstack((matrix_lon_array,matrix_lat_array)).T,dtype='float32')    
    train_y = np.array(value_list,dtype='float32')


#     OK = OrdinaryKriging(train_x[:,0], train_x[:,1], train_y, variogram_model='linear',
#                          verbose=False, nlags=50, weight=True, exact_values=False, pseudo_inv=True)
    
    OK = OrdinaryKriging(train_x[:,0], train_x[:,1], train_y, variogram_model='linear',
                         verbose=False, nlags=6, weight=True, exact_values=False, pseudo_inv=False)    

    values, ss1 = OK.execute('grid', grid_lon, grid_lat)
    plt.imshow(values)
    
    
    air_kriging.append(values)
    del matrix_lon_array; del matrix_lat_array; del train_x; del train_y; del OK; del values; del ss1
    gc.collect()
    

# print(len(air_kriging))

In [ ]:
AIR_Training = np.load('Target_Embedding/Fill_TE_Data/AIR_Align_Kriging_Training_del.npy')
AIR_Testing = np.load('Target_Embedding/Fill_TE_Data/AIR_Align_Kriging_Testing_del.npy')
sta_lat=[9,61,66,70,70,74,74,76,77,81,81,82,87,89,89,
         102,112,118,118,125,144,160,160,169,185,185,
         189,189,189,205,206,210,211,220,223,227,229,
         230,236,239,252,270,277,283,288,288,294,300,
         304,309,309,311,312,313,315,315,316,317,317,
         318,319,320,320,320,320,320,321,321,324,326,
         330,331,332]

sta_lon=[79,42,57,34,43,31,32,36,29,30,49,34,33,31,116,
         54,21,22,117,30,32,25,45,35,21,55,26,35,68,69,
         41,97,160,55,68,47,65,62,57,75,76,83,180,90,109,
         175,98,122,104,121,123,154,146,131,146,152,153,
         109,144,151,158,121,149,152,153,165,137,150,152,
         177,145,169,153]

AIR73_Training = np.zeros_like(AIR_Training)
AIR73_Testing = np.zeros_like(AIR_Testing)

for lat, lon in zip(sta_lat, sta_lon):
    AIR73_Training[:, lat, lon, :] = AIR_Training[:, lat, lon, :]
    AIR73_Testing[:, lat, lon, :] = AIR_Testing[:, lat, lon, :]

In [ ]:
total_result = []
test_x = ([[i,j] for i in range(348) for j in range(204)])

input_data = AIR73_Training


for i in tqdm(range(input_data.shape[0])):
    value_lst, result = ([] for _ in range(2))
    
    lat_all = (np.where(input_data[i,:,:,0] != 0)[0])
    lon_all = (np.where(input_data[i,:,:,0] != 0)[1])
    
    for lat, lon in zip(lat_all, lon_all): value_lst.append(input_data[i,lat,lon,0])

    train_x = np.vstack((lat_all,lon_all)).T
    train_y = value_lst
    
    
#     knn = neighbors.KNeighborsRegressor(3, weights = 'distance')
    knn = neighbors.KNeighborsRegressor(5, weights = 'distance')    
    knn_fit = knn.fit(train_x, train_y)
    test_y = knn_fit.predict(test_x)
    for k in range(348):
        result.append(test_y[k*204:(k+1)*204])
        
    total_result.append(result)

AIR73_train_fill = total_result
np.save('Target_Embedding/Fill_TE_Data/AIR73_Align_KNN5_Training.npy',AIR73_train_fill)

In [ ]:
total_result = []
test_x = ([[i,j] for i in range(348) for j in range(204)])

input_data = AIR73_Testing


for i in tqdm(range(input_data.shape[0])):
    value_lst, result = ([] for _ in range(2))
    
    lat_all = (np.where(input_data[i,:,:,0] != 0)[0])
    lon_all = (np.where(input_data[i,:,:,0] != 0)[1])
    
    for lat, lon in zip(lat_all, lon_all): value_lst.append(input_data[i,lat,lon,0])

    train_x = np.vstack((lat_all,lon_all)).T
    train_y = value_lst
    
    
#     knn = neighbors.KNeighborsRegressor(3, weights = 'distance')
    knn = neighbors.KNeighborsRegressor(5, weights = 'distance')      
    knn_fit = knn.fit(train_x, train_y)
    test_y = knn_fit.predict(test_x)
    for k in range(348):
        result.append(test_y[k*204:(k+1)*204])
        
    total_result.append(result)

AIR73_test_fill = total_result
np.save('Target_Embedding/Fill_TE_Data/AIR73_Align_KNN5_Testing.npy',AIR73_test_fill)

# SAT_Align

In [ ]:
home_path = '/data1/home/weisting/CycleGan/Experiment5/'

arrange_pd_training = pd.read_csv('old/Align_AIR_Training.csv')
SAT_time_training = np.load('/data1/home/weisting/CycleGan/Experiment4/Data_Processing/Satellite_29X17_data/SAT_pm25/an_training_time.npy',allow_pickle=True)
SAT_training = np.load(home_path+'348X204/SAT_interlinear_Training_348X204_PM25.npy',mmap_mode='r')

l1 = list(np.array(SAT_time_training,dtype='str'))
l2 = arrange_pd_training.dt.unique()
l3 = [x for x in l1 if x not in l2]
print(l3)
align_SAT = []

for i in range(len(l1)):
    if l1[i] in l3:
        print(i)
    else:
        align_SAT.append(SAT_training[i])
print('total:',len(align_SAT))

Sat_train_del = np.delete(np.array(align_SAT),[1,29,39,44,54,57,67,153,154,193,209,271,275,302,498,513,514,524,530,536,551,570,577,610,623,634,651,669,671,1091,1096,1120,1121,1124,1172,1174,1195,1223,1242,1244,1256,1262,1267,1292,1297,1306,1308,1309,1312,1313,1316,1319,1327,1335,1339,1342,1352,1358,1359,1364,1365,1366,1373,1374,1376,1386,1394,1398,1399,1401,1403,1415,1424,1430,1448,1451,1452,1453,1458,1469,1508,1527,1528,1529,1539,1556,1582,1582,1583,1623],axis=0)
print(Sat_train_del.shape)
np.save('Target_Embedding/Fill_TE_Data/SAT_Align_fill_Training_del.npy',Sat_train_del)

In [ ]:
arrange_pd = pd.read_csv('old/Align_AIR_Testing.csv')

SAT_time = np.load('/data1/home/weisting/CycleGan/Experiment4/Data_Processing/Satellite_29X17_data/SAT_pm25/an_testing_time.npy',allow_pickle=True)
SAT_testing = np.load(home_path+'348X204/SAT_interlinear_Testing_348X204_PM25.npy',mmap_mode='r')

l1 = list(np.array(SAT_time,dtype='str'))
l2 = arrange_pd.dt.unique()

l3 = [x for x in l1 if x not in l2]
print(l3)

align_SAT = []

for i in range(len(l1)):
    if l1[i] in l3:
        print(i)
    else:
        align_SAT.append(SAT_training[i])
        
Sat_test_del = np.delete(align_SAT,[159,162,164,169,206,232,236,261,272,280,315,329],axis=0)
print('total:',len(Sat_test_del))
np.save('Target_Embedding/Fill_TE_Data/SAT_Align_fill_Testing_del.npy',Sat_test_del)

In [ ]:
SAT_Training = np.load('Target_Embedding/Fill_TE_Data/SAT_Align_fill_Training_del.npy')
SAT_Testing = np.load('Target_Embedding/Fill_TE_Data/SAT_Align_fill_Testing_del.npy')

sta_lat=[9,61,66,70,70,74,74,76,77,81,81,82,87,89,89,
         102,112,118,118,125,144,160,160,169,185,185,
         189,189,189,205,206,210,211,220,223,227,229,
         230,236,239,252,270,277,283,288,288,294,300,
         304,309,309,311,312,313,315,315,316,317,317,
         318,319,320,320,320,320,320,321,321,324,326,
         330,331,332]

sta_lon=[79,42,57,34,43,31,32,36,29,30,49,34,33,31,116,
         54,21,22,117,30,32,25,45,35,21,55,26,35,68,69,
         41,97,160,55,68,47,65,62,57,75,76,83,180,90,109,
         175,98,122,104,121,123,154,146,131,146,152,153,
         109,144,151,158,121,149,152,153,165,137,150,152,
         177,145,169,153]

SAT73_Training = np.zeros_like(SAT_Training)
SAT73_Testing = np.zeros_like(SAT_Testing)

for lat, lon in zip(sta_lat, sta_lon):
    SAT73_Training[:, lat, lon, :] = SAT_Training[:, lat, lon, :]
    SAT73_Testing[:, lat, lon, :] = SAT_Testing[:, lat, lon, :]

In [ ]:
total_result = []
test_x = ([[i,j] for i in range(348) for j in range(204)])

input_data = SAT73_Training


for i in tqdm(range(input_data.shape[0])):
    value_lst, result = ([] for _ in range(2))
    
    lat_all = (np.where(input_data[i,:,:,0] != 0)[0])
    lon_all = (np.where(input_data[i,:,:,0] != 0)[1])
    
    for lat, lon in zip(lat_all, lon_all): value_lst.append(input_data[i,lat,lon,0])

    train_x = np.vstack((lat_all,lon_all)).T
    train_y = value_lst
    
#     knn = neighbors.KNeighborsRegressor(3, weights = 'distance')
    knn = neighbors.KNeighborsRegressor(5, weights = 'distance')    
    
    knn_fit = knn.fit(train_x, train_y)
    test_y = knn_fit.predict(test_x)
    for k in range(348):result.append(test_y[k*204:(k+1)*204])
        
    total_result.append(result)

Sat73_train_fill = total_result
np.save('Target_Embedding/Fill_TE_Data/SAT73_Align_KNN5_Training_del.npy',Sat73_train_fill)

In [ ]:
total_result = []
test_x = ([[i,j] for i in range(348) for j in range(204)])

input_data = SAT73_Testing


for i in tqdm(range(input_data.shape[0])):
    value_lst, result = ([] for _ in range(2))
    
    lat_all = (np.where(input_data[i,:,:,0] != 0)[0])
    lon_all = (np.where(input_data[i,:,:,0] != 0)[1])
    
    for lat, lon in zip(lat_all, lon_all): value_lst.append(input_data[i,lat,lon,0])

    train_x = np.vstack((lat_all,lon_all)).T
    train_y = value_lst
    
    
#     knn = neighbors.KNeighborsRegressor(3, weights = 'distance')
    knn = neighbors.KNeighborsRegressor(5, weights = 'distance')        
    knn_fit = knn.fit(train_x, train_y)
    test_y = knn_fit.predict(test_x)
    for k in range(348):
        result.append(test_y[k*204:(k+1)*204])
        
    total_result.append(result)

Sat73_test_fill = total_result
np.save('Target_Embedding/Fill_TE_Data/SAT73_Align_KNN5_Testing_del.npy',Sat73_test_fill)

# KNN Random Testing

In [ ]:
home_path = '/data1/home/weisting/CycleGan/Experiment5/'
EPA73_test_pm25 = np.load(home_path+'348X204/EPA73_Testing_348X204_PM25.npy', mmap_mode='r')[:3000]
station_coordinate = pd.read_csv('RandomTesting/station_coordinate')
ex5_lst = np.load('RandomTesting/extract5_list.npy')
ex10_lst = np.load('RandomTesting/extract10_list.npy')
ex15_lst = np.load('RandomTesting/extract15_list.npy')
ex20_lst = np.load('RandomTesting/extract20_list.npy')

EPA73_test_fill_ex20=[]
for qq in range(15,30):
    matrix20,lats_ex20,lons_ex20=Random_Testing(4,qq,station_coordinate, EPA73_test_pm25, ex5_lst, ex10_lst, ex15_lst, ex20_lst)

    total_result = []
    test_x = ([[i,j] for i in range(348) for j in range(204)])
    input_data = matrix20
    for i in tqdm(range(input_data.shape[0])):
        value_lst, result = ([] for _ in range(2))

        lat_all = (np.where(input_data[i,:,:,0] != 0)[0])
        lon_all = (np.where(input_data[i,:,:,0] != 0)[1])

        for lat, lon in zip(lat_all, lon_all): value_lst.append(input_data[i,lat,lon,0])

        train_x = np.vstack((lat_all,lon_all)).T
        train_y = value_lst


#         knn = neighbors.KNeighborsRegressor(3, weights = 'distance')
        knn = neighbors.KNeighborsRegressor(5, weights = 'distance')        
        
        knn_fit = knn.fit(train_x, train_y)
        test_y = knn_fit.predict(test_x)
        for k in range(348):
            result.append(test_y[k*204:(k+1)*204])

        total_result.append(result)
    EPA73_test_fill_ex20.append(total_result)
np.save('Target_Embedding/Fill_TE_Data/EPA73_testing_KNN5fill_ex20_15to30',EPA73_test_fill_ex20)

# Kriging Random Testing 

In [ ]:
home_path = '/data1/home/weisting/CycleGan/Experiment5/'
EPA73_test_pm25 = np.load(home_path+'348X204/EPA73_Testing_348X204_PM25.npy', mmap_mode='r')[:3000]
station_coordinate = pd.read_csv('RandomTesting/station_coordinate')
ex5_lst = np.load('RandomTesting/extract5_list.npy')
ex10_lst = np.load('RandomTesting/extract10_list.npy')
ex15_lst = np.load('RandomTesting/extract15_list.npy')
ex20_lst = np.load('RandomTesting/extract20_list.npy')

grid_lon = np.array(np.arange(204),dtype='float32')
grid_lat = np.array(np.arange(348),dtype='float32') 

total_mse = []
EPA73_testing_Krigingfill_ex20=[]
for qq in range(30):
    matrix20,lats_ex20,lons_ex20=Random_Testing(1,qq,station_coordinate, EPA73_test_pm25, ex5_lst, ex10_lst, ex15_lst, ex20_lst)
    total_result = []
    for i in tqdm(range(matrix20.shape[0])):
        value_lst = [] 

        lat_all = (np.where(matrix20[i,:,:,0] != 0)[0])
        lon_all = (np.where(matrix20[i,:,:,0] != 0)[1])

        for lat, lon in zip(lat_all, lon_all): value_lst.append(matrix20[i,lat,lon,0])

        matrix_lon_array = np.array(lon_all,dtype='float32')
        matrix_lat_array = np.array(lat_all,dtype='float32')       
        train_x = np.array(np.vstack((matrix_lon_array,matrix_lat_array)).T,dtype='float32')    
        train_y = np.array(value_lst,dtype='float32')

        OK = OrdinaryKriging(train_x[:,0], train_x[:,1], train_y, variogram_model='linear',
                             verbose=False, nlags=50, weight=True, exact_values=False,pseudo_inv=True)
        
#         OK = OrdinaryKriging(train_x[:,0], train_x[:,1], train_y, variogram_model='linear')        

        values, ss1 = OK.execute('grid', grid_lon, grid_lat)
        total_result.append(values)
        
    mse,mae,mape = calculate_extract_loss(np.array(total_result),EPA73_test_pm25[:,:,:,0],lats_ex20,lons_ex20)
    total_mse.append(mse)
    print(mse)        

    EPA73_testing_Krigingfill_ex20.append(total_result)
print(np.array(total_mse).mean())
# np.save('EPA73_testing_Krigingfill_ex20',EPA73_testing_Krigingfill_ex20)